In [16]:
import networkx as nx
import pandas as pd
import math

In [44]:
# These three files as well as the wanted target_ratios should be specified before calculating SLE
# Import the dataset as a csv file
asset_relationships = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students_twosets/student_export_2/haarlem_asset_relationships.csv",sep=",")
asset_types = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students_twosets/student_export_2/haarlem_asset_types.csv",sep=",")
assets = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students_twosets/student_export_2/haarlem_assets.csv",sep=",")
# Specify the wanted target ratios
target_ratios = {"OT":0.40,"IoT":0.20,"IT":0.15,"Other":0.15,"Network":0.10,"Unspecified":0.0}

In [41]:
# Functions used for the SLE calculation
# Function to get the second order neighbors counts of each node
def second_order_neighbor_counts(G):
    adj = G.adj
    result = {}
    for node in G:
        direct = set(adj[node])
        second_order = set()

        for neighbor in direct:
            for nn in adj[neighbor]:
                if nn != node and nn not in direct:
                    second_order.add(nn)
        result[node] = len(second_order)
    return result
# Function to get the PE values
def get_NPE(G,second_order_counts):
    # Clustering coefficient and cn
    c = {}
    for i in G.nodes:
        if G.degree(i) <= 1:
            c[i] = 0
        else:
            c[i] = (sum(1 for j in list(G.neighbors(i)) for k in list(G.neighbors(i)) if j != k and G.has_edge(j,k))) / (G.degree(i) * (G.degree(i) - 1))
    cn = {}
    sumcn = 0
    for i in G.nodes:
        sumneigh = len(list(G.neighbors(i))) + second_order_counts[i]
        cn[i] = sumneigh/(1+c[i])
        sumcn += cn[i]
    # I
    I = {}
    for i in G.nodes:
        I[i] = cn[i]/sumcn
    # NPE
    sum_I = {}
    NPE = {}
    for i in G.nodes:
        sum_I[i] = 0.0
        for j in G.neighbors(i):
            if I[j] == 0:
                continue
            else:
                sum_I[i] += -I[j] * math.log(I[j])
        NPE[i] = sum_I[i]
    return NPE
# Scaling NPE values based on asset type
def SLE_valuation(target_ratios,NPE,assettypes):
    NPE_df = pd.DataFrame(NPE.items(),columns=["id","value"])
    NPE_and_type = pd.merge(NPE_df,assettypes,on="id",how="left")
    total_value = NPE_and_type["value"].sum()
    group_totals = NPE_and_type.groupby("category")["value"].sum()
    scaling_factors = {t: (target_ratios[t] * total_value) / group_totals[t] for t in group_totals.index}
    NPE_and_type["adjusted_value"] = NPE_and_type.apply(lambda row: row["value"] * scaling_factors.get(row["category"],1), axis=1)
    NPE_values = NPE_and_type[["id","category","adjusted_value"]]
    return NPE_values
# Put everything in a single function
def get_SLE(assets,asset_relationships,asset_types,target_ratios):
    # Get the relationships dataframe
    relationships = asset_relationships[["source_asset_id","target_asset_id"]]
    # Get the asset_and_type dataframe
    asset_and_type = assets.merge(asset_types[["id","category"]],left_on="asset_type_id",right_on="id",how="left")
    asset_and_type["category"] = asset_and_type["category"].fillna("Unspecified")
    assettypes = asset_and_type[["id_x","category"]].rename(columns={"id_x":"id"})
    # Create the graph
    graph = nx.DiGraph()
    edges = list(zip(relationships["source_asset_id"],relationships["target_asset_id"]))
    graph.add_edges_from(edges)
    # Get the second order neighbor counts of each node
    second_order_counts = second_order_neighbor_counts(G=graph)
    # Get the NPE values of each node
    NPE = get_NPE(G=graph,second_order_counts=second_order_counts)
    # Scale based on asset type to get final SLE values
    SLE = SLE_valuation(target_ratios=target_ratios,NPE=NPE,assettypes=assettypes)
    return SLE

In [45]:
# get_SLE takes as input the assets, asset_relationship, asset_types files, as well as specified target_ratios
SLE = get_SLE(assets=assets,asset_relationships=asset_relationships,asset_types=asset_types,target_ratios=target_ratios)
SLE

,id,category,adjusted_value
0,bd5243be-e6a6-4e3d-813c-406449506692,Other,0.491421
1,00bf3e18-90fc-4cf7-bdf3-e95abc78c6cc,Unspecified,0.000000
2,823a20b3-930e-45a0-973f-d7fce9468e0b,Unspecified,0.000000
3,fe92b637-27c7-4f15-926b-a6e5ebadc1a0,Other,0.000011
4,d30b30d4-c062-4604-ae04-4185ea754998,Network,0.219728
...,...,...,...
27343,e8221cd2-4137-4423-9324-d5adb0b56cc0,Unspecified,0.000000
27344,e704412c-1e93-4a1d-9d85-46e20cf97d01,Unspecified,0.000000
27345,04ba4e72-6daf-4930-94ee-b93dee72fab7,Unspecified,0.000000
27346,971c35fe-6a14-4f34-8ea7-b874c0f52844,Unspecified,0.000000
